# IBM Applied Data Science Capstone Course by Coursera¶

## 1. Import necessary libraries

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from pandas.io.json import json_normalize

import numpy as np

import requests

from bs4 import BeautifulSoup

import json

from geopy.geocoders import Nominatim
import geocoder

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import re

print('All libraries are imported.')


All libraries are imported.


### 2. Import data from local file into a Dataframe

Download area-population related data from the Hong Kong government website (https://www.censtatd.gov.hk/hkstat/sub/sp150_tc.jsp?productCode=FA100096), and import the data into a pandas dataframe

In [93]:
csv_file = pd.read_csv('B72009FB2020XXXXB01-ªí 1 Table 1.CSV', header=2)

In [195]:
df = pd.DataFrame(csv_file)

df = df[1:21]

df

,區議會分區 District Council district,男性 Male,女性 Female,男女合計 Both sexes,男女合計 Both sexes.1,性別比率[1] Sex ratio[1],性別比率[1][2] Sex ratio[1][2]
1,香港島 Hong Kong Island,541300,687900,1229200,16.6,787,900
2,中西區 Central & Western,105900,134600,240500,3.2,787,914
3,灣仔 Wan Chai,75800,102700,178400,2.4,738,872
4,東區 Eastern,242800,302800,545600,7.4,802,898
5,南區 Southern,116800,147800,264600,3.6,790,912
6,九龍 Kowloon,1039800,1231200,2271000,30.6,845,908
7,油尖旺 Yau Tsim Mong,150200,179800,329900,4.4,835,916
8,深水埗 Sham Shui Po,191100,225400,416500,5.6,848,915
9,九龍城 Kowloon City,187600,232300,419900,5.7,808,905
10,黃大仙 Wong Tai Sin,192400,223700,416100,5.6,860,908


In [196]:
df['區議會分區 District Council district'].str.split(' ', n=1, expand=True)
df1.columns=['區議會分區','District']
df['District'] = df1['District']
df['區議會分區'] = df1['區議會分區']
df = df.drop('區議會分區 District Council district', axis=1)

df

,男性 Male,女性 Female,男女合計 Both sexes,男女合計 Both sexes.1,性別比率[1] Sex ratio[1],性別比率[1][2] Sex ratio[1][2],District,區議會分區
1,541300,687900,1229200,16.6,787,900,Hong Kong Island,香港島
2,105900,134600,240500,3.2,787,914,Central & Western,中西區
3,75800,102700,178400,2.4,738,872,Wan Chai,灣仔
4,242800,302800,545600,7.4,802,898,Eastern,東區
5,116800,147800,264600,3.6,790,912,Southern,南區
6,1039800,1231200,2271000,30.6,845,908,Kowloon,九龍
7,150200,179800,329900,4.4,835,916,Yau Tsim Mong,油尖旺
8,191100,225400,416500,5.6,848,915,Sham Shui Po,深水埗
9,187600,232300,419900,5.7,808,905,Kowloon City,九龍城
10,192400,223700,416100,5.6,860,908,Wong Tai Sin,黃大仙


In [198]:
hk_df = df[['District', '區議會分區', '男女合計 Both sexes']]
hk_df.columns = ['District','區議會分區' ,'Population']
hk_df = hk_df.reset_index(drop=True)
hk_df

,District,區議會分區,Population
0,Hong Kong Island,香港島,1229200
1,Central & Western,中西區,240500
2,Wan Chai,灣仔,178400
3,Eastern,東區,545600
4,Southern,南區,264600
5,Kowloon,九龍,2271000
6,Yau Tsim Mong,油尖旺,329900
7,Sham Shui Po,深水埗,416500
8,Kowloon City,九龍城,419900
9,Wong Tai Sin,黃大仙,416100


In [199]:
hk_df.shape

(20, 3)

### 3. Get the geographical coordinates

In [200]:
def get_latlng(district):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, HongKong'.format(district))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [203]:
coords = [ get_latlng(district) for district in hk_df["區議會分區"].tolist() ]

In [202]:
len(coords)

20

In [204]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
hk_df['Latitude'] = df_coords['Latitude']
hk_df['Longitude'] = df_coords['Longitude']
hk_df

,District,區議會分區,Population,Latitude,Longitude
0,Hong Kong Island,香港島,1229200,22.351958,114.119386
1,Central & Western,中西區,240500,22.274930,114.146239
2,Wan Chai,灣仔,178400,22.277101,114.173837
3,Eastern,東區,545600,22.272090,114.221396
4,Southern,南區,264600,22.239885,114.200435
5,Kowloon,九龍,2271000,22.311130,114.183540
6,Yau Tsim Mong,油尖旺,329900,22.309739,114.168521
7,Sham Shui Po,深水埗,416500,22.329351,114.159179
8,Kowloon City,九龍城,419900,22.326863,114.195334
9,Wong Tai Sin,黃大仙,416100,22.336088,114.191726


### 4. Create a map of HongKong

In [205]:
address = 'HongKong'

geolocator = Nominatim(user_agent='hk-explore', timeout=None)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of HongKong are {}, {}'.format(latitude, longitude))

The geographical coordinates of HongKong are 22.2793278, 114.1628131


In [207]:
map_hk = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, neighborhood in zip(hk_df['Latitude'], hk_df['Longitude'], hk_df['District']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_hk)
map_hk

In [208]:
map_hk.save('map_hk.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [233]:
CLIENT_ID = '13TMJMMA0X1YCDRQR5YJJSIWBE4GVYUHQQWWUFUNTYIESLBJ'
CLIENT_SECRET = 'QULCZZJIZ3VZQIH3F3RTOH35AHWMVGPFKW1GM5X4XK0HXS0C'
VERSION = '20201128'

In [240]:
radius = 1000
LIMIT = 100

venues = []

for lat, lng, district in zip(hk_df['Latitude'], hk_df['Longitude'], hk_df['District']):
    url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lng,
    radius,
    LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues.append((
        district,
        lat,
        lng,
        venue['venue']['name'],
        venue['venue']['location']['lat'],
        venue['venue']['location']['lng'],
        venue['venue']['categories'][0]['name']            
        ))

In [241]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['District', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1155, 7)


,District,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Hong Kong Island,22.351958,114.119386,Kwai Tsing Theatre (葵青劇院),22.356319,114.126540,Theater
1,Hong Kong Island,22.351958,114.119386,Shanghai Po Po (上海婆婆),22.357137,114.126996,Shanghai Restaurant
2,Hong Kong Island,22.351958,114.119386,LOG-ON,22.357667,114.126352,Gift Shop
3,Hong Kong Island,22.351958,114.119386,Broadway Cinema (百老匯戲院),22.357438,114.126728,Multiplex
4,Hong Kong Island,22.351958,114.119386,Chiuchow Garden (潮庭),22.357649,114.126743,Chinese Restaurant


In [242]:
venues_df['VenueCategory'].unique()[:50]

array(['Theater', 'Shanghai Restaurant', 'Gift Shop', 'Multiplex',
       'Chinese Restaurant', 'Café', 'Fast Food Restaurant', 'Platform',
       'Metro Station', 'Cha Chaan Teng', 'Restaurant',
       'Convenience Store', 'Park', 'Japanese Restaurant', 'Trail',
       'Garden', 'Scenic Lookout', 'Playground', 'Road',
       'Monument / Landmark', 'Waterfall', 'Indian Restaurant',
       'Seafood Restaurant', 'Art Gallery', 'Vietnamese Restaurant',
       'Italian Restaurant', 'Comedy Club', 'Coffee Shop',
       'Asian Restaurant', 'Bistro', 'Ice Cream Shop', 'Butcher',
       'Pizza Place', 'Bakery', 'Fish & Chips Shop', 'Hotel',
       'Dessert Shop', 'Sushi Restaurant', 'Art Museum', 'Supermarket',
       'Shopping Mall', 'Tram Station', 'Noodle House', 'Tree',
       'Watch Shop', 'Bubble Tea Shop', 'Science Museum', 'Grocery Store',
       'Bus Stop', 'Thai Restaurant'], dtype=object)

In [243]:
venues_df['VenueCategory'].loc[venues_df['VenueCategory'].str.contains('Restaurant')]='Restaurant'

In [244]:
venues_df['VenueCategory'].value_counts()

Restaurant                       471
Coffee Shop                       65
Dessert Shop                      52
Café                              45
Noodle House                      45
Shopping Mall                     41
Cha Chaan Teng                    38
Hotel                             27
Park                              19
Supermarket                       16
Market                            14
Train Station                     12
Convenience Store                 11
Snack Place                       11
Athletics & Sports                10
Burger Joint                      10
Bakery                            10
Pizza Place                        9
Department Store                   9
Clothing Store                     8
Sandwich Place                     7
Bus Station                        7
Bus Stop                           6
Multiplex                          6
Steakhouse                         6
Scenic Lookout                     6
BBQ Joint                          5
T

In [245]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 136 unique categories.


### 6. Analyze Each Neighborhood

In [246]:
hk_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

hk_onehot['District'] = venues_df["District"]

fixed_columns = [hk_onehot.columns[-1]] + list(hk_onehot.columns[:-1])
hk_onehot = hk_onehot[fixed_columns]

print(hk_onehot.shape)
hk_onehot.head()

(1155, 137)


,District,ATM,Arcade,Art Gallery,Art Museum,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,Boutique,Bowling Alley,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cha Chaan Teng,Chinese Breakfast Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hockey Field,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Imported Food Shop,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Light Rail Station,Lounge,Market,Massage Studio,Metro Station,Monument / Landmark,Movie Theater,Multiplex,Non-Profit,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racecourse,Record Shop,Recreation Center,Rest Area,Restaurant,Road,Sandwich Place,Scenic Lookout,Science Museum,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Taxi Stand,Tea Room,Temple,Tennis Court,Theater,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Watch Shop,Waterfall,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Hong Kong Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Hong Kong Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hong Kong Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hong Kong Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hong Kong Island,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [248]:
hk_grouped = hk_onehot.groupby(['District']).mean().reset_index()

print(hk_grouped.shape)
hk_grouped

(20, 137)


,District,ATM,Arcade,Art Gallery,Art Museum,Astrologer,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,Boutique,Bowling Alley,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cha Chaan Teng,Chinese Breakfast Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Event Space,Farmers Market,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Historic Site,History Museum,Hobby Shop,Hockey Field,Hostel,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Imported Food Shop,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Light Rail Station,Lounge,Market,Massage Studio,Metro Station,Monument / Landmark,Movie Theater,Multiplex,Non-Profit,Noodle House,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racecourse,Record Shop,Recreation Center,Rest Area,Restaurant,Road,Sandwich Place,Scenic Lookout,Science Museum,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Taxi Stand,Tea Room,Temple,Tennis Court,Theater,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Watch Shop,Waterfall,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Central & Western,0.000000,0.000000,0.018519,0.018519,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.018519,0.018519,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.092593,0.018519,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.018519,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.018519,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.037037,0.000000,0.00,0.018519,0.018519,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.018519,0.000000,0.000000,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.018519,0.00,0.000000,0.00,0.018519,0.000000,0.018519,0.000000,0.00,0.00,0.000000,0.000000,0.018519,0.000000,0.018519,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.222222,0.018519,0.000000,0.074074,0.018519,0.018519,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018519,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.055556,0.000000,0.018519,0.018519,0.018519,0.018519,0.00,0.000000,0.000000,0.000000
1,Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.055556,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.388889,0.000000,0.000000,0.000000,0.000000,0.055556,0.00,0.055556,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Hong Kong Island,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [279]:
len(hk_grouped[hk_grouped["Restaurant"] > 0])

18

In [285]:
hk_restaurant = hk_grouped[["District","Restaurant"]]
hk_restaurant = hk_restaurant.join(hk_df.set_index('District'), on='District')
hk_restaurant['Restaurant'] = hk_restaurant['Restaurant'].astype(float)
hk_restaurant['Population'] =hk_restaurant['Population'].astype(float)
hk_restaurant['Restaurant per Person'] = hk_restaurant['Restaurant']/hk_restaurant['Population']
hk_restaurant = hk_restaurant.drop('區議會分區', axis=1)
hk_restaurant

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person
0,Central & Western,0.222222,240500.0,22.274930,114.146239,9.240009e-07
1,Eastern,0.388889,545600.0,22.272090,114.221396,7.127729e-07
2,Hong Kong Island,0.421053,1229200.0,22.351958,114.119386,3.425420e-07
3,Kowloon,0.480000,2271000.0,22.311130,114.183540,2.113606e-07
4,Kowloon City,0.420000,419900.0,22.326863,114.195334,1.000238e-06
5,Kwai Tsing,0.347826,502400.0,22.349591,114.117464,6.923290e-07
6,Kwun Tong,0.428571,688500.0,22.314236,114.226625,6.224712e-07
7,New Territories,0.457627,3922300.0,22.445410,114.026640,1.166732e-07
8,North,0.000000,314100.0,22.508196,114.300653,0.000000e+00
9,Sai Kung,0.000000,472500.0,22.400012,114.313797,0.000000e+00


### 7. Cluster Neighborhoods

In [286]:
kclusters = 3

hk_clustering = hk_restaurant.drop(["District"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hk_clustering)

kmeans.labels_[0:10]

array([0, 0, 2, 2, 0, 0, 0, 1, 0, 0], dtype=int32)

In [305]:
hk_merged = hk_restaurant.copy()

hk_merged["Cluster Labels"] = kmeans.labels_
hk_merged

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
0,Central & Western,0.222222,240500.0,22.274930,114.146239,9.240009e-07,0
1,Eastern,0.388889,545600.0,22.272090,114.221396,7.127729e-07,0
2,Hong Kong Island,0.421053,1229200.0,22.351958,114.119386,3.425420e-07,2
3,Kowloon,0.480000,2271000.0,22.311130,114.183540,2.113606e-07,2
4,Kowloon City,0.420000,419900.0,22.326863,114.195334,1.000238e-06,0
5,Kwai Tsing,0.347826,502400.0,22.349591,114.117464,6.923290e-07,0
6,Kwun Tong,0.428571,688500.0,22.314236,114.226625,6.224712e-07,0
7,New Territories,0.457627,3922300.0,22.445410,114.026640,1.166732e-07,1
8,North,0.000000,314100.0,22.508196,114.300653,0.000000e+00,0
9,Sai Kung,0.000000,472500.0,22.400012,114.313797,0.000000e+00,0


In [306]:
hk_merged.rename(columns={"District": "District"}, inplace=True)
hk_merged.head()

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
0,Central & Western,0.222222,240500.0,22.274930,114.146239,9.240009e-07,0
1,Eastern,0.388889,545600.0,22.272090,114.221396,7.127729e-07,0
2,Hong Kong Island,0.421053,1229200.0,22.351958,114.119386,3.425420e-07,2
3,Kowloon,0.480000,2271000.0,22.311130,114.183540,2.113606e-07,2
4,Kowloon City,0.420000,419900.0,22.326863,114.195334,1.000238e-06,0


In [309]:
print(hk_merged.shape)
hk_merged.sort_values(["Cluster Labels"], inplace=True)
hk_merged

(20, 7)


,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
0,Central & Western,0.222222,240500.0,22.274930,114.146239,9.240009e-07,0
19,Yuen Long,0.483333,645000.0,22.445131,114.025732,7.493540e-07,0
18,Yau Tsim Mong,0.279070,329900.0,22.309739,114.168521,8.459223e-07,0
1,Eastern,0.388889,545600.0,22.272090,114.221396,7.127729e-07,0
4,Kowloon City,0.420000,419900.0,22.326863,114.195334,1.000238e-06,0
5,Kwai Tsing,0.347826,502400.0,22.349591,114.117464,6.923290e-07,0
6,Kwun Tong,0.428571,688500.0,22.314236,114.226625,6.224712e-07,0
8,North,0.000000,314100.0,22.508196,114.300653,0.000000e+00,0
9,Sai Kung,0.000000,472500.0,22.400012,114.313797,0.000000e+00,0
11,Sham Shui Po,0.420000,416500.0,22.329351,114.159179,1.008403e-06,0


In [310]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(hk_merged['Latitude'], hk_merged['Longitude'], hk_merged['District'], hk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 8. Exam the cluster

Cluster 0

In [311]:
hk_merged.loc[hk_merged['Cluster Labels'] == 0]

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
0,Central & Western,0.222222,240500.0,22.274930,114.146239,9.240009e-07,0
19,Yuen Long,0.483333,645000.0,22.445131,114.025732,7.493540e-07,0
18,Yau Tsim Mong,0.279070,329900.0,22.309739,114.168521,8.459223e-07,0
1,Eastern,0.388889,545600.0,22.272090,114.221396,7.127729e-07,0
4,Kowloon City,0.420000,419900.0,22.326863,114.195334,1.000238e-06,0
5,Kwai Tsing,0.347826,502400.0,22.349591,114.117464,6.923290e-07,0
6,Kwun Tong,0.428571,688500.0,22.314236,114.226625,6.224712e-07,0
8,North,0.000000,314100.0,22.508196,114.300653,0.000000e+00,0
9,Sai Kung,0.000000,472500.0,22.400012,114.313797,0.000000e+00,0
11,Sham Shui Po,0.420000,416500.0,22.329351,114.159179,1.008403e-06,0


Cluster 1

In [312]:
hk_merged.loc[hk_merged['Cluster Labels'] == 1]

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
7,New Territories,0.457627,3922300.0,22.44541,114.02664,1.166732e-07,1


Cluster 2

In [313]:
hk_merged.loc[hk_merged['Cluster Labels'] == 2]

,District,Restaurant,Population,Latitude,Longitude,Restaurant per Person,Cluster Labels
3,Kowloon,0.480000,2271000.0,22.311130,114.183540,2.113606e-07,2
2,Hong Kong Island,0.421053,1229200.0,22.351958,114.119386,3.425420e-07,2


Most restaurants are concentrated in the city center of Hong Kong, with the largest number in cluster 0. The number of cluster 2 and 1 is very small. This is an excellent opportunity and high potential area to open a new restaurant, because there is almost no competition in the heart. At the same time, due to oversupply and high concentration of restaurants, restaurants in cluster 0 are likely to suffer fierce competition. From another perspective, this also shows that the oversupply of restaurants mainly occurs in the center of the city, while there are still few shopping centers in the suburbs. Therefore, when there is little or no competition, to open new restaurants in cluster 1 and cluster 2, it is recommended that real estate developers avoid communities in cluster 0 that already have a large number of restaurants and are highly competitive.